In [1]:
import random
import mne, os, glob
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import random
%matplotlib qt 

In [2]:
"""
***Aestetics***
"""
# load color palette
my_palette = sns.color_palette().as_hex()
sns.color_palette().as_hex()

['#1f77b4',
 '#ff7f0e',
 '#2ca02c',
 '#d62728',
 '#9467bd',
 '#8c564b',
 '#e377c2',
 '#7f7f7f',
 '#bcbd22',
 '#17becf']

In [10]:
""" 
Load files
"""

var_el_path = "C:/Users/mvmigem/Documents/data/project_1/overlap_corrected/variable-electrode/"
fix_el_path = "C:/Users/mvmigem/Documents/data/project_1/overlap_corrected/fixed-electrode/"

var_dir_list = glob.glob(var_el_path+'/*.csv')
fix_dir_list = glob.glob(fix_el_path+'/*.csv')
excuded_pp = [3,14,20]
subject_list = []

for i in var_dir_list:
    sub = int(i.split('corrected_evoked_')[1].split('.csv')[0])
    if sub not in excuded_pp:
        subject_list.append(sub)

# Create big df for each var and fix set of rERP's
var_loaded_dfs = []
fix_loaded_dfs = []

for var_p,fix_p in zip(var_dir_list,fix_dir_list):
    var_rerp_df = pd.read_csv(var_p)
    var_loaded_dfs.append(var_rerp_df)
    fix_rerp_df = pd.read_csv(fix_p)
    fix_loaded_dfs.append(fix_rerp_df)

var_df = pd.concat(var_loaded_dfs,ignore_index=True)
fix_df = pd.concat(fix_loaded_dfs,ignore_index=True)
var_df['visual_field'] = np.where(var_df['position'].isin([1,2]),'up','down')
fix_df['visual_field'] = np.where(fix_df['position'].isin([1,2]),'up','down')


In [27]:
""" 
basline correction
"""
baseline_period = var_df['time'] <0
baseline_means = var_df[baseline_period].groupby(['subject','attention','expectation','visual_field'])['yhat'].mean()

var_df['baseline_corrected'] = var_df.apply(
    lambda row: row['yhat'] - baseline_means[(row['subject'], row['attention'], row['expectation'],row['visual_field'])],
    axis=1
)


In [32]:
fig, axes = plt.subplots()

sns.lineplot(data=var_df[var_df['visual_field']=='down'],x='time',y='baseline_corrected',hue='attention',style='expectation')
plt.show()